<a href="https://colab.research.google.com/github/hellomada/RayField-Systems-Internship-Task-week-2-4/blob/final_app/Final_App_For_Rayfield_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install streamlit pandas scikit-learn matplotlib joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install streamlit pyngrok


In [ ]:
!pip install streamlit pyngrok scikit-learn matplotlib pandas joblib


In [ ]:

# Set your ngrok auth token (replace with your actual token)
!ngrok authtoken 2qFGMWOMeEuHTiLPxw8jEmXaNpb_6eCNm53johjExqNGmDuo4

# Write the Streamlit app code to app.py
app_code = """
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import joblib
import io
import os

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score

def feature_engineering(df):
    X = df[['hour', 'dc_power', 'daily_yield', 'total_yield']]
    return X

def train_models(x_train, y_train):
    lr = LinearRegression()
    lr.fit(x_train, y_train)

    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(x_train, y_train)

    gbr = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gbr.fit(x_train, y_train)

    adr = AdaBoostRegressor(n_estimators=100, random_state=42)
    adr.fit(x_train, y_train)

    lr2 = LinearRegression()
    lr2.fit(x_train, y_train)

    ensemble = VotingRegressor([
        ("GBR", gbr),
        ("RFR", rf),
        ("ADR", adr),
        ("LR", lr2)
    ])
    ensemble.fit(x_train, y_train)

    return ensemble

MODEL_FILENAME = "ensemble_model.joblib"

def main():
    st.set_page_config(page_title="Energy AI Dashboard", layout="wide")
    st.title("⚡ Energy AI Dashboard with AI Model & Anomaly Detection")

    uploaded_file = st.file_uploader("Upload energy CSV with columns ['date', 'hour', 'dc_power', 'daily_yield', 'total_yield', 'ac_power']", type=["csv"])

    if uploaded_file:
        try:
            df = pd.read_csv(uploaded_file)
        except Exception as e:
            st.error(f"Error reading CSV: {e}")
            return

        required_cols = ['date', 'hour', 'dc_power', 'daily_yield', 'total_yield', 'ac_power']
        if not all(col in df.columns for col in required_cols):
            st.error(f"CSV must have columns: {required_cols}")
            return

        df['date'] = pd.to_datetime(df['date'])

        X = feature_engineering(df)
        y = df['ac_power']

        if os.path.exists(MODEL_FILENAME):
            model = joblib.load(MODEL_FILENAME)
            st.info("Loaded pre-trained ensemble model.")
        else:
            with st.spinner("Training model..."):
                model = train_models(X, y)
                joblib.dump(model, MODEL_FILENAME)
                st.success("Model trained and saved.")

        df['predicted_ac_power'] = model.predict(X)

        iso = IsolationForest(contamination=0.05, random_state=42)
        df['anomaly'] = iso.fit_predict(df[['predicted_ac_power']])
        df['anomaly'] = df['anomaly'] == -1

        anomaly_count = df['anomaly'].sum()
        summary = f"This week, {anomaly_count} anomalies detected in predicted AC power."

        col1, col2 = st.columns([3,2])

        with col1:
            st.subheader("Energy Output Chart")
            fig, ax = plt.subplots(figsize=(10,5))
            ax.plot(df['date'], df['ac_power'], label="Actual AC Power", color='blue')
            ax.plot(df['date'], df['predicted_ac_power'], label="Predicted AC Power", color='green', linestyle='--')
            anomalies = df[df['anomaly']]
            ax.scatter(anomalies['date'], anomalies['predicted_ac_power'], color='red', label='Anomalies', zorder=5)
            ax.set_xlabel("Date")
            ax.set_ylabel("AC Power")
            ax.legend()
            ax.set_title("Actual vs Predicted AC Power with Anomalies")
            st.pyplot(fig)

        with col2:
            st.subheader("Weekly Summary")
            st.markdown(f"**{summary}**")

            st.subheader("Anomaly Alerts")
            if anomaly_count == 0:
                st.write("No anomalies detected.")
            else:
                for _, row in anomalies.iterrows():
                    st.write(f"⚠️ Anomaly on {row['date'].date()}: Predicted AC Power = {row['predicted_ac_power']:.2f}")

            if anomaly_count > 0:
                csv_buffer = io.StringIO()
                anomalies[['date', 'predicted_ac_power']].to_csv(csv_buffer, index=False)
                st.download_button(
                    label="Download Anomalies CSV for Zapier",
                    data=csv_buffer.getvalue(),
                    file_name="anomalies_today.csv",
                    mime="text/csv"
                )

if __name__ == "__main__":
    main()
"""

with open("app.py", "w") as f:
    f.write(app_code)

from pyngrok import ngrok
import threading
import time

# Kill previous tunnels
ngrok.kill()

# Open tunnel on port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit app URL: {public_url}")

# Run Streamlit app in background thread
def run_streamlit():
    !streamlit run app.py

threading.Thread(target=run_streamlit, daemon=True).start()

# Wait a few seconds for Streamlit server to start
time.sleep(5)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app URL: NgrokTunnel: "https://0b1e13a6e4d5.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.8.147:8501



In [ ]:
!streamlit run app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.185.8.147:8502

  Stopping...
  Stopping...
  Stopping...
Exception ignored in: <module 'threading' from '/usr/lib/python3.11/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1541, in _shutdown
    def _shutdown():
    
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 43, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/server/server.py", line 509, in stop
    self._runtime.stop()
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/runtime.py", line 329, in stop
    async_objs.eventloop.call_soon_threadsafe(stop_on_eventloop)
  File "/usr/lib/python3.11/asyncio/base_events.py", line 807, in call_soon_threadsafe
    self._check_closed()
  File "/usr/lib/python3.11/asyncio/base_events.py",

In [ ]:
if __name__ == "__main__":
    main()

2025-08-13 03:27:06.603 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 03:27:06.609 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 03:27:07.064 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-13 03:27:07.067 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 03:27:07.070 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 03:27:07.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-13 03:27:07.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn